<a href="https://colab.research.google.com/github/rjthompson22/MBSUS/blob/main/Copy_of_Ultrasound_Brightness_Profiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Libraries to interact with google drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive


#Libraries to get Brightness Profiles
import os
import cv2
import numpy as np
import pandas as pd

print("Complete")

Complete


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
print("Complete")

Mounted at /content/gdrive
Complete


In [ ]:
in_dir = '/content/gdrive/My Drive/MBSUS/Videos To Be Analyzed'
out_dir = '/content/gdrive/My Drive/MBSUS/Brightness Profile Analyses'
frame_dir = out_dir + "/Video Frames"
print("Complete")

Complete


# Convert Video into Frames

In [ ]:
%cd /content/gdrive/My Drive/MBSUS/Brightness Profile Analyses/Video Frames

for vid in os.listdir(in_dir):
  #print(str(vid)[:-4])
  vidObj = cv2.VideoCapture(in_dir +"/"+ vid) 
  time_stamp = vidObj.get(cv2.CAP_PROP_POS_MSEC)
	# Used as counter variable 
  count = 0

	# checks whether frames were extracted 
  success = 1

  while success: 

		# vidObj object calls read 
		# function extract frames 
    success, image = vidObj.read() 

		# Saves the frames with frame-count 
    if success != 1: break
        
    cv2.imwrite(str(vid)[:-4] + "_frame%d.jpg" % count, image)

    count += 1
        
  vidObj.release()
  cv2.destroyAllWindows()

  print("Complete")

/content/gdrive/My Drive/MBSUS/Brightness Profile Analyses/Video Frames
P03-3-RLT
Complete


# Create Brightness Profiles of Video Frames

In [ ]:
import matplotlib.pyplot as plt
plt.close("all")

shape0 = 500

for file in os.listdir(frame_dir):
    fn = os.path.join(frame_dir, file)
    im = cv2.imread(fn)
    rrate = im.shape[0] / shape0
    shape1 = int(im.shape[1] / rrate)
    im = cv2.resize(im, (shape1, 500))
    im = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)

    out_shape0 = im.shape[0]
    out_shape1 = (im.shape[1] - 60) // 10
    if out_shape1 * 10 + 60 < im.shape[1]:
        out_shape1 += 1

    out = np.zeros((out_shape0, out_shape1))
    for r in range(out_shape0):
        for i in range(out_shape1):
            ret = np.mean(im[r, 40 + i * 10: 50 + i * 10])
            out[r, i] = ret

    im = cv2.cvtColor(im, cv2.COLOR_GRAY2RGB)
    for i in range(out_shape1):
      i *= 10

      #Add Lines
      cv2.line(im, (i+40, 0), (i+40, shape0), (0, 255, 255), 1)
      cv2.line(im, (i+50, 0), (i+50, shape0), (0, 255, 255), 1)

      #Save changes
      #cv2.imwrite(str(file), im)

    cls = np.array(["" for i in range(out_shape1)])
    dpt = np.array([str(i + 1) for i in range(out_shape1)])
    out = np.vstack([cls, dpt, out])

    ind = np.expand_dims(np.array(["Classification", "Depth(pixels)"] + [str(i + 1) for i in range(out_shape0)]), 1)
    out1 = np.hstack([ind, out])

    df = pd.DataFrame(out1)
    df1 = pd.DataFrame(out[1:])

    #Create an excel spreadsheet
    #df.to_excel(os.path.join(out_dir, file + ".xlsx"), header=False, index=False)

    #Create graphs
    df1 = df1.astype(float)


    figure, axes = plt.subplots(1, 2)
    axes[0].imshow(im)
    df1.loc[:,len(df1.columns)/2].plot(ax=axes[1], legend = None)
    #plt.show()

    plt.savefig(str(file)[:-4]+'_fig.png')
    os.remove(file)
    cv2.destroyAllWindows()

print("Complete")



# Create Video Out Of Brightness Profiles

In [ ]:
import cv2
import numpy as np

img_array = []
for filename in os.listdir(frame_dir):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
    os.remove(filename)
    cv2.destroyAllWindows()

out = cv2.VideoWriter("FINDME.mp4",0x7634706d, 10, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()